# Load Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install polars==1.25.2 >> _

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-polars-cu12 25.2.2 requires polars<1.22,>=1.20, but you have polars 1.25.2 which is incompatible.


In [3]:
!pip install implicit >> _

In [4]:
!pip install mlflow >> _

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
cudf-polars-cu12 25.2.2 requires polars<1.22,>=1.20, but you have polars 1.25.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [5]:
!pip install numpy >> _
!pip install scipy >> _

In [37]:
!pip install lightfm >> _

In [6]:
!mkdir data

In [7]:
# # takes 5 minutes
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/clickstream.pq -O data/clickstream.pq >> _
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/test_users.pq -O data/test_users.pq >> _
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/cat_features.pq -O data/cat_features.pq >> _
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/text_features.pq -O data/text_features.pq >> _
!wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/events.pq -O data/events.pq >> _


--2025-05-17 16:31:25--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/clickstream.pq
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 967808682 (923M) [application/octet-stream]
Saving to: ‘data/clickstream.pq’

data/clickstream.pq 100%[===================>] 922.97M  20.3MB/s    in 48s     

2025-05-17 16:32:14 (19.4 MB/s) - ‘data/clickstream.pq’ saved [967808682/967808682]

--2025-05-17 16:32:14--  https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/test_users.pq
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaitin

In [38]:
from datetime import timedelta
import polars as pl
import implicit
import itertools
from lightfm import LightFM

In [9]:
import mlflow
from scipy.sparse import csr_matrix
import numpy as np
import time

In [10]:
DATA_DIR = '/kaggle/working/data'

df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

# PREPARE TRAIN EVAL

In [11]:
EVAL_DAYS_TRESHOLD = 14

In [12]:
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)

In [13]:
df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]

In [14]:
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')


In [15]:
df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

In [16]:
df_eval = df_eval.filter(
        pl.col('cookie').is_in(df_train['cookie'].unique())
    ).filter(
        pl.col('node').is_in(df_train['node'].unique())
    )

In [17]:
df_train

cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458
1,14233689,17,2025-01-16 19:36:23,2,2,214458
…,…,…,…,…,…,…
149999,21815295,17,2025-01-29 10:31:16,2,2,58872
149999,4486457,17,2025-01-29 13:38:32,2,11,51162
149999,4486457,17,2025-01-29 10:41:51,2,11,51162


# TRAIN MODEL

## ALS

In [18]:
def get_als_pred(users, nodes, user_to_pred, 
                 factors=60, 
                 iterations=10, 
                 regularization=0.1, 
                 alpha=1.0,
                 use_cg=True,
                 random_state=None):
    
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    model = implicit.als.AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=regularization,
        alpha=alpha,
        use_cg=use_cg,
        random_state=random_state
    )
    
    model.fit((sparse_matrix * alpha).astype('float64'))
    
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    recommendations, scores = model.recommend(
        user4pred, 
        sparse_matrix[user4pred], 
        N=40, 
        filter_already_liked_items=True
    )
    
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ], 
            'cookie': list(user_to_pred),
            'scores': scores.tolist()
        }
    )
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred

In [43]:
def get_als_model(users, nodes, user_to_pred, 
                 factors=60, 
                 iterations=10, 
                 regularization=0.1, 
                 alpha=1.0,
                 use_cg=True,
                 random_state=None):
    
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    model = implicit.als.AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=regularization,
        alpha=alpha,
        use_cg=use_cg,
        random_state=random_state
    )
    
    model.fit((sparse_matrix * alpha).astype('float64'))
    return model

In [19]:
users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes,eval_users )

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

## popular

In [20]:
def get_popular(df):
    popukar_node = df.group_by('node').agg(pl.col('cookie').count()).sort('cookie').tail(40)['node'].to_list()
    df_pred_pop = pl.DataFrame({'node': [popukar_node for i in range(len(eval_users))], 'cookie': eval_users})
    df_pred_pop = df_pred_pop.explode('node')
    return df_pred_pop

train_pop = get_popular(df_train)


# CALC EVAL METRICS

In [21]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()


In [22]:
recall_at(df_eval, df_pred, k=40)

0.15352464317402645

In [23]:
recall_at(df_eval, train_pop, k=40)

0.05989061839535582

# SUMBIT

In [24]:
# users = df_clickstream["cookie"]
# nodes = df_clickstream["node"]
# test_users = df_test_users['cookie'].unique().to_list()

# df_pred = get_als_pred(users, nodes, test_users )


In [25]:
# df_pred.write_csv('prediction.csv')

# MLflow

In [26]:
mlflow.set_tracking_uri("http://51.250.35.156:5000/")

In [27]:
# mlflow.create_experiment(
#     name="homework-avikornelyuk.ext",
#     artifact_location="mlflow-artifacts:/"
# )

In [48]:
mlflow.set_experiment(experiment_name="homework-avikornelyuk.ext")

<Experiment: artifact_location='/mlflow-artifacts', creation_time=1747428315965, experiment_id='55', last_update_time=1747431701764, lifecycle_stage='active', name='homework-avikornelyuk.ext', tags={'mlflow.note.content': "Лучший запуск: модель ALS, параметры: `'factors': "
                        "100, 'iterations': 10, 'regularization': 0.001, "
                        "'alpha': 1, 'use_cg': True`\n"
                        '\n'
                        'GitHub: '}>

In [29]:
users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

# Поиск параметров ALS

In [ ]:
param_grid_als = {
    'factors': [50, 100],
    'iterations': [10, 15],
    'regularization': [0.005, 0.01],
    'alpha': [0.5, 1, 5],
    'use_cg': [True, False],
    'random_state': [42]    
}

param_names_als = list(param_grid_als.keys())
param_value_lists_als = list(param_grid_als.values())

all_param_combinations_als = list(itertools.product(*param_value_lists_als))

print(f"Комбинаций: {len(all_param_combinations_als)}")

for i, param_values_tuple_als in enumerate(all_param_combinations_als):
    current_als_params = dict(zip(param_names_als, param_values_tuple_als))

    run_name_als = f"ALS_f{current_als_params['factors']}" \
                   f"_i{current_als_params['iterations']}" \
                   f"_reg{current_als_params['regularization']}" \
                   f"_alpha{current_als_params['alpha']}" \
                   f"_cg{'T' if current_als_params['use_cg'] else 'F'}"

    print(f"\nЗАПУСК {i+1}/{len(all_param_combinations_als)}: {run_name_als}")
    print(f"Параметры: {current_als_params}")

    with mlflow.start_run(run_name=run_name_als):
        mlflow.log_params(current_als_params)
        mlflow.log_param("model_type", "ALS")
        
        start_time_train = time.time()
        
        df_pred = get_als_pred(
            users, nodes, eval_users, **current_als_params
        )
        
        training_time = time.time() - start_time_train
        mlflow.log_metric("training_time_sec", training_time)
        print(f"Время обучения: {training_time:.2f} сек.")

        if df_pred is not None and df_pred.height > 0:
            r_als = recall_at(df_eval, df_pred, k=40)
            mlflow.log_metric("Recall_40", r_als)
            print(f"Recall@40: {r_als:.4f}")
                            
        elif df_pred is None:
             print("df_pred is None.")
             mlflow.log_metric("Recall_40", 0.0)
        else:
            print("df_pred пустой")
            mlflow.log_metric("Recall_40", 0.0)

### Результат лучшей модели

In [65]:
als_params = {
    'factors': 100,
    'iterations': 10, 
    'regularization': 0.001,
    'alpha': 1,
    'use_cg': True,
    'random_state': 42
}

with mlflow.start_run(run_name="ALS_best_Run"):
    mlflow.log_params(als_params)
    mlflow.log_param("model_type", "ALS")
    df_pred = get_als_pred(users, nodes, eval_users, **als_params)
    
    model = get_als_model(users, nodes, eval_users, **als_params)
    model.save("als_model.npz")
    mlflow.log_artifact("als_model.npz", artifact_path="mlflow-artifacts")

    r_als = recall_at(df_eval, df_pred, k=40)
    mlflow.log_metric("Recall_40", r_als)
    print(f"ALS Recall@40: {r_als:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

ALS Recall@40: 0.1580
🏃 View run ALS_best_Run at: http://51.250.35.156:5000/#/experiments/55/runs/08705c1aa891448c9f5f82a19936fa93
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55


# Поиск параметров BPR 

In [31]:
from implicit.bpr import BayesianPersonalizedRanking

In [41]:
def get_bpr_pred(users, nodes, user_to_pred, factors=60, iterations=10, learning_rate=0.01, regularization=0.1, random_state=None):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    model = BayesianPersonalizedRanking(
        factors=factors,
        iterations=iterations,
        learning_rate=learning_rate,
        regularization=regularization,
        random_state=random_state
    )
    model.fit(sparse_matrix)
    
    user4pred_indices = np.array([user_id_to_index[i] for i in user_to_pred if i in user_id_to_index]) # Убедимся, что юзеры есть в словаре
    valid_user_to_pred = [i for i in user_to_pred if i in user_id_to_index]

    if len(user4pred_indices) == 0:
         return pl.DataFrame({'node': [], 'cookie': [], 'scores': []})


    recommendations, scores = model.recommend(
        user4pred_indices, 
        sparse_matrix[user4pred_indices], 
        N=40, 
        filter_already_liked_items=True
    )
    
    df_pred = pl.DataFrame({
        'node': [[index_to_item_id.get(i) for i in rec_list if index_to_item_id.get(i) is not None] for rec_list in recommendations.tolist()],
        'cookie': valid_user_to_pred,
        'scores': scores.tolist()
    })
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred, model

In [42]:
param_grid_bpr = {
    'factors': [100, 200],
    'iterations': [10, 20],
    'learning_rate': [0.005, 0.05],
    'regularization': [0.001, 0.1],
    'random_state': [42]
}
param_names = list(param_grid_bpr.keys())
param_value_lists = list(param_grid_bpr.values())

all_param_combinations = list(itertools.product(*param_value_lists))
print(f"Комбинаций: {len(all_param_combinations)}")

for i, param_values_tuple in enumerate(all_param_combinations):
    current_bpr_params = dict(zip(param_names, param_values_tuple))

    run_name = f"BPR_factors_{current_bpr_params['factors']}" \
               f"_iter_{current_bpr_params['iterations']}" \
               f"_lr_{current_bpr_params['learning_rate']}" \
               f"_reg_{current_bpr_params['regularization']}"

    print(f"\nЗАПУСК {i+1}/{len(all_param_combinations)}: {run_name}")
    print(f"Параметры: {current_bpr_params}")

    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(current_bpr_params)

        start_time = time.time()
        df_pred_bpr, bpr_model_instance = get_bpr_pred(
            users, nodes, eval_users, **current_bpr_params
        )
        training_time = time.time() - start_time
        mlflow.log_metric("training_time_sec", training_time)

        if df_pred_bpr.height > 0:
            r_bpr = recall_at(df_eval, df_pred_bpr, k=40)
            mlflow.log_metric("Recall_40", r_bpr)
            print(f"BPR Recall@40: {r_bpr:.4f} (Обучение: {training_time:.2f} сек)")

        else:
            print("Для этого набора параметров предсказания не сгенерированы.")
            mlflow.log_metric("Recall_40", 0.0)
            mlflow.log_metric("training_time_sec", training_time)

Комбинаций: 16

ЗАПУСК 1/16: BPR_factors_100_iter_10_lr_0.005_reg_0.001
Параметры: {'factors': 100, 'iterations': 10, 'learning_rate': 0.005, 'regularization': 0.001, 'random_state': 42}


  0%|          | 0/10 [00:00<?, ?it/s]

BPR Recall@40: 0.0920 (Обучение: 113.53 сек)
🏃 View run BPR_factors_100_iter_10_lr_0.005_reg_0.001 at: http://51.250.35.156:5000/#/experiments/55/runs/c0941821a32a4603b38562fb15a54bd8
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55

ЗАПУСК 2/16: BPR_factors_100_iter_10_lr_0.005_reg_0.1
Параметры: {'factors': 100, 'iterations': 10, 'learning_rate': 0.005, 'regularization': 0.1, 'random_state': 42}


  0%|          | 0/10 [00:00<?, ?it/s]

BPR Recall@40: 0.0679 (Обучение: 111.41 сек)
🏃 View run BPR_factors_100_iter_10_lr_0.005_reg_0.1 at: http://51.250.35.156:5000/#/experiments/55/runs/ab4d59fd718948faba6f5ae7867273ef
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55

ЗАПУСК 3/16: BPR_factors_100_iter_10_lr_0.05_reg_0.001
Параметры: {'factors': 100, 'iterations': 10, 'learning_rate': 0.05, 'regularization': 0.001, 'random_state': 42}


...

BPR Recall@40: 0.0699 (Обучение: 174.02 сек)
🏃 View run BPR_factors_200_iter_20_lr_0.005_reg_0.1 at: http://51.250.35.156:5000/#/experiments/55/runs/33556c66a5a34967ae8dbf005d46f169
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55

ЗАПУСК 15/16: BPR_factors_200_iter_20_lr_0.05_reg_0.001
Параметры: {'factors': 200, 'iterations': 20, 'learning_rate': 0.05, 'regularization': 0.001, 'random_state': 42}


  0%|          | 0/20 [00:00<?, ?it/s]

BPR Recall@40: 0.0679 (Обучение: 178.70 сек)
🏃 View run BPR_factors_200_iter_20_lr_0.05_reg_0.001 at: http://51.250.35.156:5000/#/experiments/55/runs/e221fdc6f34c44f394b0bb114bcd454b
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55

ЗАПУСК 16/16: BPR_factors_200_iter_20_lr_0.05_reg_0.1
Параметры: {'factors': 200, 'iterations': 20, 'learning_rate': 0.05, 'regularization': 0.1, 'random_state': 42}


  0%|          | 0/20 [00:00<?, ?it/s]

BPR Recall@40: 0.0094 (Обучение: 177.40 сек)
🏃 View run BPR_factors_200_iter_20_lr_0.05_reg_0.1 at: http://51.250.35.156:5000/#/experiments/55/runs/23e8339f7d724742ae868bc5405876bb
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55


# Поиск параметров LightFM

In [39]:
def get_lightfm_pred(
    users, 
    items,
    user_to_pred,
    factors=64,
    epochs=10,
    learning_rate=0.05,
    loss='warp',
    random_state=42,
    N=40,
):
    user_ids = users.unique().to_list()
    item_ids = items.unique().to_list()
    u2i = {u: idx for idx, u in enumerate(user_ids)}
    i2i = {i: idx for idx, i in enumerate(item_ids)}
    i2item = {idx: i for i, idx in i2i.items()}

    rows = users.to_list()
    cols = items.to_list()
    row_idx = [u2i[r] for r in rows]
    col_idx = [i2i[c] for c in cols]
    data = np.ones(len(row_idx), dtype=np.float32)
    interactions = coo_matrix(
        (data, (row_idx, col_idx)),
        shape=(len(user_ids), len(item_ids))
    )

    model = LightFM(
        no_components=factors,
        learning_rate=learning_rate,
        loss=loss,
        random_state=random_state
    )
    model.fit(interactions, epochs=epochs, num_threads=4)

    preds = []
    for u in user_to_pred:
        if u not in u2i:
            continue
        ui = u2i[u]
        scores = model.predict(ui, np.arange(len(item_ids)))
        top_idx = np.argsort(-scores)[:N]
        for idx in top_idx:
            preds.append({
                'cookie': u,
                'node': i2item[idx],
                'scores': float(scores[idx])
            })

    df_pred = pl.DataFrame(preds)
    return df_pred, model


In [40]:
param_grid_lfm = {
    'factors': [32, 64],
    'epochs': [10, 20],
    'learning_rate': [0.01, 0.05],
    'loss': ['warp', 'bpr'],
    'random_state': [42]
}

param_names = list(param_grid_lfm.keys())
param_value_lists = list(param_grid_lfm.values())

all_param_combinations = list(itertools.product(*param_value_lists))
print(f"Комбинаций: {len(all_param_combinations)}")

for vals in itertools.product(*param_grid_lfm.values()):
    params = dict(zip(param_grid_lfm.keys(), vals))
    run_name = f"LFM_f{params['factors']}_e{params['epochs']}_lr{params['learning_rate']}_{params['loss']}"
    
    print(f"\nЗАПУСК {i+1}/{len(all_param_combinations)}: {run_name}")
    print(f"Параметры: {params}")
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(params)
        t0 = time.time()
        df_pred_lfm, lfm_model = get_lightfm_pred(
            users, nodes, eval_users, **params, N=40
        )
        mlflow.log_metric("training_time_sec", time.time() - t0)
        if df_pred_lfm.height:
            r = recall_at(df_eval, df_pred_lfm, k=40)
        else:
            r = 0.0
        mlflow.log_metric("Recall_40", r)
        # mlflow.lightfm.log_model(lfm_model, artifact_path="lightfm_model")
        print(f"{run_name} → Recall@40 = {r:.4f}")


Комбинаций: 16

ЗАПУСК 1/16: LFM_f32_e10_lr0.01_warp
Параметры: {'factors': 32, 'epochs': 10, 'learning_rate': 0.01, 'loss': 'warp', 'random_state': 42}
🏃 View run LFM_f32_e10_lr0.01_warp at: http://51.250.35.156:5000/#/experiments/55/runs/476d64935e4e4df68da780b0331249c6
🧪 View experiment at: http://51.250.35.156:5000/#/experiments/55


KeyboardInterrupt: 